<a href="https://colab.research.google.com/github/christopherhuntley/DATA6510/blob/master/L7_Dimensional_DW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/christopherhuntley/DATA6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **DATA 6510**
# **Lesson 7: Dimensional Data Warehouses** 
_Facts, Dimensions, and Cubes_

## **Learning Objectives**
### **Theory / Be able to explain ...**
- How database design for analytical applications is different from that for transaction systems
- The various forms of the star schema and when each is most applicable
- The concept of data granularity in dimensional data warehouses
- Data cubes as a dimensional data model
- Recent SQL extensions geared towards analytical applications 

### **Skills / Know how to ...**
- Identify dimensions that provide context to facts
- Use SQL window functions to work with longitudinal data
- Use SQL arrays and structs to work with repeating groups and hierarchical data structures

--------

## **BIG PICTURE: The Holy Grail of Data Warehousing**

Data Warehousing as the "one true source of truth" is an idea that goes all the way back to the 1980s. Analysts had by that point built up quite a repertoire of models for just about any kind of analysis. They could create classification and regression trees (decision trees, random forests, etc.). They could do linear, nonlinear, kernel, and logistic regression with large datasets. They could solve optimization problems with thousands of variables and tens of thousands of constraints. Even the neural network models at the core of the latest and greatest deep learning techniques were pretty mature by 1994.  

What was missing was data! Well, sort of. Mainframe systems had been collecting transaction data for decades. Every day, banks and credit card companies were processing millions of transactions, transportation systems were tracking hundreds of thousands of shipments, etc. All we had to do was archive it all so we could analyze it later. Or so it seemed to most of us. 

Analytical data is not the same as transactional data, of course, in ways that were clear at the time.   

|   | Transactional Data | Analytical Data |
|---| | |
| **Scope** | Functional / Operational | Strategic / Executive |
| **Provenance** | Online / Production | Historical + 3-rd Party |
| **Operating Environment** | Enterprise / Big Iron | Workspace / Workstation |
| **Data Quality** | "good enough to run the company" | "small errors compound over time" |
| **Performance Objective** | maximize transactions per hour | minimize time to results |
| **Access to Datasets** | read-only queries and reports | offline, with ability to make corrections |

Where would anybody get the analytical data with the right scope, volume, quality, etc. quickly and easily enough to useful? 

Even back then the ultimate solutions were known, though not remotely close to being available. Consider, for example, this figure from Ralph Kimball's seminal book *The Data Warehouse Lifecycle Toolkit*, published in 1998 and based on original work **from the mid-1980s**.

![Kimball's Data Warehouse](https://github.com/christopherhuntley/DATA6510/raw/master/img/L10_Kimball_Data_Warehouse_Elements.png)

All of the elements of a modern data pipeline are there. It even articulated the steps of the ETL process in detail. It was all there ... to be realized *someday*. 

Someday is now. With commodity data storage, ample computing power, ready-made software for just about any kind of modeling, and the analytical results to attract attention from management, data infrastructure is finally seen as what it should have been all along: a critical resource upon which the company relies to make it stand out against the competition. 

That's what the vendors tell us, anyway. 

In this lesson we will explore the **Dimensional Data Warehouse** model first proposed by Kimball et al. all those years ago. We will also consider how its mass adoption has influenced the SQL standard in recent years, with the addition of **window functions** and **collection** data types that relax fundamental assumptions of the relational data model in favor of analytical use cases. 

---
## **The Star Schema Pattern**

At some level, nearly all data warehouses look roughly the same. There is a huge table in the center with lots of columns and foreign key relationships to  smaller tables around the periphery. This general pattern is called a **star schema**. A relational database that implements the star schema pattern (or one of its variants) is called a dimensional data warehouse.

> **Heads Up**: The star schema is a **design pattern, a standard solution to a standard problem.** By standardizing what they call these solutions, designers can communicate among themselves with just a few words instead of re-explaining the details every time. It is both more efficient and potentially less error-prone, as any standard should be.   

The Star Schema pattern features two kinds of tables:
- **A fact table with precomputed quantitative measures.** The measures themselves are somewhat volatile, with new measures continually added and others redefined to suit the ever changing needs of the analysts. If there is a way to precompute a statistic or other measure so analysts don't have to, then do it. If a given measure is no longer needed or misleading, then we redefine or remove it. 
- **Dimension tables that provide context for the facts.** Dimensions are somewhat timeless and immutable. Even when the facts themselves may change over time, the dimensions remain relatively static. 

Generally, the fact table has a foreign key reference to each of the dimension tables. Dimension tables, meanwhile, stand on their own, without any foreign keys. 

The star schema addresses the disconnect between the way data is recorded versus how it is used by analysts. It reduces all data down to measures (facts) and context (dimensions). Since *all information* takes that form eventually, star schemas strike a nice balance between structure and general applicability.

Once again, here is the NBA PlayFacts warehouse, this time noting some of the key features. We will use it as an example, starting with the dimensions before moving on to the facts. We will also explore variations on the general star schema pattern that fit certain use cases. 
![NBA PlayFacts Dim DW](https://github.com/christopherhuntley/DATA6510/raw/master/img/L10_Star_Schema_Notes.png)

---
## **Dimension Tables**
Dimensions are the lens through which we interpret each fact. They are what give it context and meaning. 

In theory the **dimensions are strong entities that exist independently of the facts**. Each fact, meanwhile, represents a collection (more like a selection) of details found in the dimensions. We'll go deeper into this when get discuss fact tables and granularity. 

Though there is some disagreement about this, **the usual recommendation is that dimension tables be fully _denormalized_**. Since they are often fairly small (relative to the fact table) and don't change much, there is little chance of creating anomalies over time. So, while it may be tempting to, for example, normalize out zip codes and cities from a location dimension, there is no real need, especially when it would require an unnecessary table join.

So, what kinds of details are we talking about? A good starting point is the framework used by journalists and storytellers the world over:
- **Who was involved?** People, roles, etc. 
- **What happened** Event types, outcomes, etc. 
- **When did it happen?** Timing or place in a sequence
- **Where did it happen?** Location, which may be conceptual rather than physical
- **Why did it happen?** Intent, cause, etc. 
- **How did it happen?** Steps, sequential logic, etc. 


Each of the who/what/when/where/why/how questions can suggest multiple dimensions. As we can see in the NBA example, the question of "who is involved in a given play?" is answered with *three* dimensions: 
- the individual player who gets credited with each event
- the lineup of players on the court at the time
- the team whose play is being reflected by the facts

It is done this way to support different, independently-calculated measures:
- the counting stats (points, rebounds, assists, etc.) for an individual player, lineup, or team
- the total playing time for each player or lineup  on the court, even when they are not generating counting stats

### **How Many Dimensions? Granularity and Focus**
An often overlooked but potentially tricky aspect of dimensional design is whether dimensions are allowed to overlap. In other words, can the same dimension be represented two different ways? Can we combine dimensions to create a third uber-dimension? 

Like a lot of things, it depends.  

**When to combine dimensions:** Some dimensions may permit several levels of *granularity*. For example, a given office is in a region, which is in a division, etc. To borrow terminology from normalization, we say that there is a chain of functional dependencies: office $\rightarrow$ region $\rightarrow$ division. The recommendation is to keep the levels together in a single dimension table rather than separate them out. 
> This would apply, for example, to the way NBA teams are grouped into *divisions* and *conferences* (Eastern or Western) that affect scheduling decisions. In other words, team $\rightarrow$ division $\rightarrow$ conference. Since each group nests cleanly inside the next, there is no need to separate them. 

**When to separate into multiple dimensions:** Within a company the *location* of a fact can mean geolocation (addresses) or a spot in organizational hierarchy (functional area, group, etc.). While they are both location dimensions, they are not logically connected via a functional dependency. They should reside in separate dimension tables, not one. 
> We have already seen this case with the NBA players, lineups, and teams. There are many-to-many relationships among them, making it impossible to create a consistent functional dependency chain; they don't nest together cleanly. 

### **Slowly Changing Dimensions**

There are plenty of specialized types of dimensions (conformed, role-playing, junk, etc.) that serve different purposes. Most of the time, however, we are working with so-called **_slow moving_** dimensions where the rows and columns don't change much over time. Often such dimensions can be maintained by hand or perhaps via a periodic updating process. 

For example, while NBA players do occasionally move from team to team or sign "10 day contracts" to replace injured players, for the most part the team rosters do not change much during the season. We can maintain them as needed, starting with a preliminary roster at the start of the seaosn and then adding to it as new players join the team. 

Similarly, NBA games are scheduled released well in advance, allowing us to update the games dimension all at once with perhaps a rare game reschedule if needed. 

---
## **Fact Tables**
**Fact tables exist at the intersection of the dimension tables.** Each fact is labeled with foreign keys, usually one key per dimension. The rest of the columns are measures that can be used in aggregate calculations.  

What makes a good measure? Anything for which we can calculate descriptive statistics (counts, averages, etc.): 
- For text data, we generally are limited to the text itself and counts of some sort. We may, for example, count the number of times the word "no" appears, how many sentences there are, etc. 
- For numerical data we can use all of the usual statistics like mean, maximum, minimum, etc., or perhaps *bin* into nonoverlapping categories (or *segments*). 
- For temporal data (dates and times), we may calculate elapsed times, inter-event times, cumulative times, etc. that can be treated like numerical data.
- For binary data (pictures, etc.), the options are very limited, though one may be able to apply a machine learning technique to generate numerical digests that can be aggregated.  

Interestingly, the fact table can only be as **granular** as the **dimensions** allow. In other words, if a given dimension only has 3 possible labels (rows), then that dimension can only divide up the facts three ways. We can, however, increase the granularity by adding **new** dimensions. Each additional dimension potentially increases the granularity but never decreases it. Alternatively, we can increase the granularity by adding strategically selected rows to an **existing** dimension. 

One way to visualize this is with a (hyper-) cube, with each dimension on a side. Each fact is *binned* inside one of the smaller cubes at the intersection of the dimensions. For the NBA PlayFacts cube below, each fact is binned based on the game, team, and player. Thus, if with only three dimensions, we would only be able to generate box score stats for full games. In order to get statistics within a game (e.g., for the last two minutes of each period) we would need to include a *play segment* dimension. (Don't ask about how we'd show a 4-dimensional cube. Just know that we can.) Thus, by adding a dimension we have also increased the granularity. 

![Data Cube](https://github.com/christopherhuntley/DATA6510/raw/master/img/L10_DataCube_wide.png)

> **Heads Up:** It is sometimes difficult to distinguish dimensions from measures when source data is numerical. For example, is the time on the clock (i.e., seconds remaining in the period) a measure or a dimension? It is a measure, in the sense that it captures the passage of time, but it is also a dimension, in that it records when a given event happened. The key when considering whether any given quantity belongs on the fact table is to ask whether you would i) aggregate it (sum, average, etc.) or ii) cite it. In a basketball game it is the latter (cited), so we separate it out into the play segment dimension. The clock *interval* between events (elapsed time), however, is something that we can sum up by quarter, player, etc. Thus, it belongs on the fact table.  

### **Rollups and Drilldowns**

One of the big advantages of a dimensional data warehouse design is that it makes it very simple to aggregate and disaggregate data at various levels of granularity. 

> **Heads Up**: The terms "rollup" and "drilldown" were coined by vendors of OLAP (OnLine Analytical Processing) systems that do real-time ETL from transaction data but the terms can be readily applied to any dimensional data warehouse.

A **rollup** is the standard `GROUP BY` aggregation operation. The idea is that a whole stripe of data in the cube (i.e., a dimension) is "rolled up" like a carpet and then replaced with summary data. We can do this for several dimensions at a time to get summarized data of various purposes. 

A **drilldown** is the opposite of a rollup. Starting with aggregate data, a drilldown disaggregates it to a finer level of granularity. Behind the scenes, it is the same as a rollup, just with a lower level of aggregation. Ultimately, the lowest level of a drilldown is the fact table itself. 
 


---
## **Variations**
### **Dimension *Tables* vs Dimension *Columns***
One of the advantages of keeping dimensional data in separate tables is that it can significantly reduce storage costs by eliminating redundant data. However, with the advent of cheap cloud-based data storage, cost becomes less important than performance. Thus, we may choose to denormalize everything into a single table (or materialized view) that doesn't require any expensive joins. It's simple enough. If we already have the data in a normalized form, then we would just need to join in every table and select every column to generate a new "one table fits all" data warehouse. 

```sql
-- A materialzied view is like a self-updating table
CREATE MATERIALIZED VIEW denormalized_fact_table AS
  ( SELECT *
    FROM fact_table 
      JOIN dimension_a ...
      JOIN dimension_b ...
      JOIN ...
  );
```
The dimensions would still be there, just as columns instead of tables. The joins would be completed in advance, simplifying `SELECT` queries even further.

> Note that in practice we would not use * in a view object; it's much more efficient to list specific columns. Also, materialized views need to be refreshed just before use to avoid stale data. 

While denormalized views sound great in theory, there are two good reasons for creating separate dimension tables: 
- **If care is taken to *use just foreign keys in the `GROUP BY` clause*, then it can *sometimes* be actually be faster to query multiple tables than a single table.** This is because the joins will happen **after** the grouping has reduced the data to a manageable number of fact table rows. The incremental performance cost of the join is then practically nil, especially if there are a small number of groups in the result set. 
- **Dimension tables provide opportunities to add in static descriptive data.** For example, we could add in the seating capacity or age of a given basketball arena if we treat it as a dimension table instead of just a column. If an arena were just a few columns then we'd have to update the view each time we added a column. 

Whether either of these advantages are relevant depends on the situation. As a general rule, unless you have a good reason not to, it is best to create dimension tables instead of dimension columns. You can always create a view if needed. 

### **Snowflakes and Galaxies**

The rule that dimensions be denormalized is more of a convenience than a law. The purpose is to make writing queries as simple and bug-free as possible. However, if we are ultimately going to denormalize the data using the same joins every time, then what difference does it make how many joins there are? Such is the reasoning behind the so-called **Snowflake design pattern**. 

Here, for example, is the snowflake version of the PlayFacts database. The dotted lines show relationships normalized out from the star schema dimensions. There is no effect on the fact table, just more detail in the dimensions. 
![](https://github.com/christopherhuntley/DATA6510/raw/master/img/L10_NBA_PlayFacts_Snowflake_DW.png) 

The advantages of normalizing the dimensions include:
- less data redundancy, smaller storage requirements, etc. that we usually associate with normalization
- the ability to add new details (like team franchises or arenas) without disrupting existing dimensions

It is important to point out that the snowflake and star schema patterns are both perfectly valid. Which to use is totally situational. Let the data, its usage, and performance considerations guide your decision. 

Like the snowflake pattern, the **galaxy pattern** also extends the star schema, this time by allowing multiple fact tables. The baseball database from lesson 2 is a galaxy with ...
- Fact tables for fielding, batting, pitching, etc.  
- Dimension tables for players (Master), teams, all star game appearances, etc. 
It also follows the snowflake pattern, with enhancements like All Star Game appearances, Hall of Fame voting, etc. attached to the players and teams. 

![Lahman 2016 ERD](https://github.com/christopherhuntley/DATA6510/raw/master/img/L2_baseball_stats_schema.png)


---
## **Advanced SQL Tips**

### **Window Functions**

Longitudinal data, where the meaning of a given fact is determined relative to the ones before it, is inherently problematic. The complexity is subtle, practically invisible to untrained eyes, but it can nonetheless wreak havoc on even the best data model. 

The relational data model is designed to capture instantaneous snapshots of whatever is being tracked, adding one row at a time. Similarly, `SELECT` queries return data one row at a time. That makes it nice and symmetric, with rows in and rows out. 

Now for the subtle part. If we want to include a calculated column, say the difference between two other columns in the same row, then we'd do something like this:
```sql
SELECT column_b - column_a AS b_minus_a 
FROM ... ;
```
That seems simple enough. We can also, of course, use `GROUP BY` to calculate subtotals within a given column if we like:
```sql
SELECT sum(column_a) as subtotal_a
FROM ...
GROUP BY column_b;
```
In both cases we are performing a calculation over *nonoverlapping* groups of rows and columns (i.e., either within each row or aggregated within predefined groups of rows).

However, what if we need to calculate differences between each row and *the one above it*? Then it gets more complicated, *because the groups overlap*. Not even `GROUP BY` can help us with that, unless we somehow make each row and the one above it a group, which seems highly inefficient at best. We'll instead have to dig deeper into our bag of tricks: 
- Join a table to itself using some sort of lag logic on the keys:
```
SELECT t2.column_a - t1.column_a AS diff_a_lag_1
FROM table AS t1 JOIN table AS t2 ON (t1.id = t2.id + 1)
```
- Use Python or another programming language to do the work. pandas has the `shift()` method just for such a purpose. You merge one table with a shifted version of itself. If you go this route, of course, you will need SQL to read the input data and then write the calculations back to the database afterwards. That's a nontrivial solution to say the least.  
- Use a SQL **window function**, which operates vertically within a column instead of laterally within a row. 

For most use cases, window functions will be your best choice. 

Window functions are used in the `SELECT` and `ORDER BY` clauses to aggregate longitudinal data *without a `GROUP BY` clause*. Instead of the traditional grouping operation, the expression being calculated specifies a *sliding window* of rows, anchored by the current row. The syntax is 
```sql
expression OVER ( window_selector )
```
The `window_selector` specifies what rows to include and can use pretty sophisticated selection criteria like any of the following:
- X rows before and Y rows after the current row
- All rows before the current row
- All rows where column X is within some range of Z in the current row
- X rows before and Y rows after, sorted by the value of Z

The window function partitions the rows (much like `GROUP BY`) based on the given criteria and then calculates the expression based on the current row. The expression is almost always an aggregation function like `SUM()` or `AVG()`, of which there are too many to list here. There are even special ones like [`STRING_AGG()`](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#string_agg) that work with non-numeric data types. 

Here, for example, we are calculating the cumulative minutes for each NBA player lineup throughout a season of play.  





In [ ]:
# load the bigquery magics extension
%load_ext google.cloud.bigquery
%load_ext google.colab.data_table

# authenticate for GCP access
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
Authenticated


In [ ]:
%%bigquery --project nba-lineup-facts
SELECT 
    game_id, 
    date,
    lineup, 
    sum(play_length_mins) OVER 
      (PARTITION BY lineup ORDER BY game_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) 
      AS cum_mins
FROM `nba-lineup-facts.lineup_facts.game_facts_all` 
WHERE year = 2020
ORDER BY lineup, date
LIMIT 100

,game_id,date,lineup,cum_mins
0,21900415,2019-12-18,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",0.083333
1,21900665,2020-01-22,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",0.700000
2,21900679,2020-01-24,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",4.233333
3,21900757,2020-02-05,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",13.450000
4,21900757,2020-02-05,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",1.733333
...,...,...,...,...
95,21900602,2020-01-14,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",2.666667
96,21900588,2020-01-12,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",1.500000
97,21900602,2020-01-14,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",2.183333
98,21900076,2019-11-01,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",3.216667


The window selector has three parts:
```sql 
PARTITION BY lineup 
ORDER BY game_id 
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
```
- `PARTITION BY` works like a `GROUP BY` to split the rows into groups/partitions.
- `ORDER BY` does as expected, but always with respect to the partition.
- `ROWS` specifies the width of the window; in this case that means everything up to the current row in the current partition (lineup).

There are lots of variations to play with. For details read the docs for your database vendor. The SQL standard defines the basics like `PARTITION BY`, `ORDER BY`, and `ROWS` but there are some differences for more advanced use cases.  

> **Heads Up**: Although window functions were included in SQL:2003, most of the vendors only recently implemented the standard.  Check to see if your DBMS supports them before trying to use them.  

---
### **Collection Types**

In lesson 5 we learned that entities should be atomic, indivisible through normalization. Among other things this means that attributes should be singular, represeting just one fact about a given entity. Plural attributes, meanwhile, should be normalized out into one-to-many relationships to new entity types. 

But what if we just didn't bother to do that? 

The `ARRAY` and `STRUCT` collection types (details below) were added to the SQL standards in a wave of enthusiasm for object-relational features in SQL:1999 but then (like window functions) largely ignored by most database vendors until 2018. Why ignore the standard? Because collection types permit us to skirt around a fundamental law of normalization: thou shalt not use repeated fields. 

In fact, it is not a good idea to use these sorts of collection types in a transactional database, where vigilance is needed to keep the data consistent and anomaly free. However, for dimensional data warehouses, where the **dimension tables** are often denormalized anyway, they make **perfect sense**: no extra joins, no redundant columns, etc., just the relevant context in a concise format.  

> **Heads Up**: Collection types are generally **not a good fit for fact tables** because they complicate the aggregation logic. What does a **measure** (i.e., fact) mean when it can be multi-valued? How do you sum or average a list of lists? 

***Technically*, collection types do not actually violate normalization rules.** A collection is a **container** that can hold structured data. The container itself is singular, though the contents might be plural. It's like how you might keep a plastic *container* of cut celery sticks instead of storing them in a pile on the shelf. The container is *one* thing (with the sticks nested inside), but you have to open the container (unpack it) to get your snack. That's exactly how SQL `ARRAY` and `STRUCT` work. We have to use functions and special syntax to pack and unpack data. Otherwise, SQL just treats it like an image or other binary data. Of course, it *is* still a problem that we don't know what's inside the container before we unpack it but we can still say the table itself is 1NF. 

Now let's go back to the NBA data to see how it works. To keep things simple we will just focus on the lineup dimension, which in our play log data is kept in columns `a1`-`a5` and `h1`-`h5`.

   


### **The `ARRAY` Data Type**

An `ARRAY` is a list of items where:
- The items are ordered, one after another, like the rows of a table.  
- All items have the same data type. If the first item is an integer, then so is every other item. 
- There can be any number of items. 

Note that these same rules also apply to table columns, where each column has a data type, is ordered by row, and there is no limit on how many items (rows) there are. This is by design, so that the same logic can apply.  

The query below returns two rows of play log data, with each lineup as an **`ARRAY`** (represented by the square brackets `[ ]`)  

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT
  [a1,a2,a3,a4,a5] as away_lineup, 
  [h1,h2,h3,h4,h5] as home_lineup
FROM `nba-lineup-facts.lineup_facts.play_log_sample`
WHERE play_id IN (86,88);

,away_lineup,home_lineup
0,"[OG Anunoby, Marc Gasol, Kyle Lowry, Fred VanV...","[Gordon Hayward, Jaylen Brown, Kemba Walker, R..."
1,"[OG Anunoby, Marc Gasol, Kyle Lowry, Fred VanV...","[Gordon Hayward, Kemba Walker, Robert Williams..."


At first glance, this looks a lot like the text strings we have been using to represent NBA lineups. To see the difference, consider these two queries.


In [ ]:
%%bigquery --project nba-lineup-facts

-- Lineup string: "['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']"
-- Does the lineup string include the text 'Marc Gasol'?
SELECT "['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']" LIKE '%Marc Gasol%'

,f0_
0,True


In [ ]:
%%bigquery --project nba-lineup-facts

-- Lineup ARRAY: ['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']
-- Does the player 'Marc Gasol' appear in the lineup array? 
SELECT 'Marc Gasol' IN UNNEST(['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet'])

,f0_
0,True


Both return the same result but work very differently. The first query looks for the text `'Marc Gasol'` within a lineup string, while the second query looks for *Marc Gasol's name* in the lineup array. That's much more natural, isn't it? Further, the array version is much more flexible. We could, for example, replace every player's name with a `player_id` without generating and searching through lineup strings. 

There is another difference as well. When retrieving an `ARRAY` as a list we use the `UNNEST` function, which gets around the relational model's rule that each column contains scalar (singular) values. The stored array is a *container* for list items that needs to be *unpacked* before we can use it. The container is a singular thing that happens to hold more things.

> **Heads Up**: We can also [create arrays from subqueries using the `ARRAY()` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/array_functions), which opens up lots of possibilities. Basically, any column of data that we can select or construct can be inserted into an array.  
 

### **The `STRUCT` Data Type**

A `STRUCT` (short for Structured Array) is like a one-row mini-table, with named *fields* that can mix data of various types. Some call it the swiss army knife of data types. Anything we can store in a table row (including `ARRAY` data) can also be stored in a structured array.

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT
  STRUCT(a1 AS p1, a2 AS p2, a3 AS  p3,a4 AS p4,a5 AS p5) AS away_lineup,
  STRUCT(h1 AS p1, h2 AS p2, h3 AS  p3,h4 AS p4,h5 AS p5) AS home_lineup
FROM `nba-lineup-facts.lineup_facts.play_log_sample`
WHERE play_id IN (86,88);

,away_lineup,home_lineup
0,"{'p1': 'OG Anunoby', 'p2': 'Marc Gasol', 'p3':...","{'p1': 'Gordon Hayward', 'h2': 'Jaylen Brown',..."
1,"{'p1': 'OG Anunoby', 'p2': 'Marc Gasol', 'p3':...","{'p1': 'Gordon Hayward', 'h2': 'Kemba Walker',..."


Each `STRUCT` works more or less like a `SELECT` clause:
- The fields are defined like column specs, with aliases used to name the fields.
- `STRUCT` collections are created by calling a function, `STRUCT()`, that assembles the data into the structured array.
- Each field can store the result of any SQL expression. The expression is only evaluated once, just before the data is written to the field.

Again, a structured array is like a one-row table, with all the sophistication that implies. All it is missing is primary and foreign keys.

> **Heads Up**: The notation used by SQL to represent a structured array is basically identical to the Python `dict` and the JavaScript `object`. We will explore the relationship to JavaScript more in depth in Lesson 11. 

To unpack a structured array we use `column.field` dot notation. The example below packs players into home and away lineups (via a CTE `WITH` clause) and then unpacks them to recreate the `h1`-`h5` and `a1`-`a5` columns in the original play log dataset. 


In [ ]:
%%bigquery --project nba-lineup-facts
-- A named subquery with two STRUCT columns
WITH play_lineups AS  
  ( SELECT
      play_id,
      STRUCT(a1 AS p1, a2 AS p2, a3 AS  p3,a4 AS p4,a5 AS p5) AS away_lineup,
      STRUCT(h1 AS p1, h2 AS p2, h3 AS  p3,h4 AS p4,h5 AS p5) AS home_lineup
    FROM `nba-lineup-facts.lineup_facts.play_log_sample`
    WHERE play_id IN (86,88)
  )
-- Use the subquery like a table
SELECT home_lineup.p1 as h1, home_lineup.p2 as h2, home_lineup.p3 as h3, home_lineup.p4 as h4, home_lineup.p5 as h5,
       away_lineup.p1 as a1, away_lineup.p2 as a2, away_lineup.p3 as a3, away_lineup.p4 as a4, away_lineup.p5 as a5
FROM play_lineups


,h1,h2,h3,h4,h5,a1,a2,a3,a4,a5
0,Gordon Hayward,Jaylen Brown,Kemba Walker,Robert Williams III,Marcus Smart,OG Anunoby,Marc Gasol,Kyle Lowry,Fred VanVleet,Norman Powell
1,Gordon Hayward,Kemba Walker,Robert Williams III,Marcus Smart,Grant Williams,OG Anunoby,Marc Gasol,Kyle Lowry,Fred VanVleet,Norman Powell


### **Combining `ARRAY` and `STRUCT` to make complex data types**

We can combine `ARRAY` and `STRUCT` data to create arbitrarily complex data structures. The result is **trees** of data, with collections nested inside collections inside of collections ...

We saw an example in Lesson 5 when we explored JSON formatted data. The JSON fields shown below with arrowheads are collections, which have even more collections nested inside:
![](https://github.com/christopherhuntley/DATA6510/raw/master/img/L6_Space_Jam_JSON.png)

The actual JSON data, with nested arrays and structs can be found [here](https://github.com/christopherhuntley/DATA6510/blob/master/data/JSON/spacejam.json).

To show how this is done in SQL, we will explore two techniques for creating *pseudo*-tables using nested collections:
- Create each column as an `ARRAY` and then nesting them inside a `STRUCT`
- Create each row as a `STRUCT` and then nesting them inside an `ARRAY`

In both cases the result is a single value that is functionally equivalent to an entire table. 

> **Heads Up**: The examples below are not intended to be of much practical use. There are likely better ways to store tabular data, like perhaps in a table. They do, however, show the flexibility of the collection data types. Any data we can store in a table we can also store in a collection. 









#### **Nesting `ARRAY` columns inside a `STRUCT` pseudo-table**

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT STRUCT(
  ARRAY(SELECT play_id FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as play_id,
  ARRAY(SELECT a1 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a1,
  ARRAY(SELECT a2 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a2,
  ARRAY(SELECT a3 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a3,
  ARRAY(SELECT a4 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a4,
  ARRAY(SELECT a5 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a5,
  ARRAY(SELECT a1 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h1,
  ARRAY(SELECT a2 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h2,
  ARRAY(SELECT a3 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h3,
  ARRAY(SELECT a4 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h4,
  ARRAY(SELECT a5 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h5 
) AS pseudo_table_by_cols

,pseudo_table_by_cols
0,"{'play_id': [86, 88], 'a1': ['OG Anunoby', 'OG..."


#### **Nesting `STRUCT` rows inside an `ARRAY` pseudo-table**

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT ARRAY(
  SELECT STRUCT(play_id, a1,a2,a3,a4,a5, h1,h2,h3,h4,h5)
  FROM `nba-lineup-facts.lineup_facts.play_log_sample`
  WHERE play_id in (86,88) 
) AS pseudo_table_by_rows

,pseudo_table_by_rows
0,"[{'play_id': 86, 'a1': 'OG Anunoby', 'a2': 'Ma..."


---
## **Congratulations! You've made it to the end of Lesson 7.**

Next week we will consider alternative data models that can improve flexibility and performance. 



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `DATA6510` folder so you can find it next time.